In [220]:
import requests
from bs4 import BeautifulSoup
import os
import time

In [233]:
class scraper:
    
    def __init__(self):
        self.requests_headers = {
        "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; ; NCLIENT50_AAP5EF3046787B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
        }


class dogdrip_scrap(scraper):
    
    def __init__(self,scrap_type = 0, page = 1):
        
        """page : 페이지 넘버, 
        scrap_type : 스크래핑 url 선택, 0 : 개드립 인기글, 1 : 개드립,
        2 : 유저 개드립 인기글, 3 : 유저 개드립
        index_type : 게시판 별 첫번째 글 순서, 공지가 추가될 시 변할 수 있음.
        """
        
        super().__init__() # requests_headers
        url_list = [f'https://www.dogdrip.net/index.php?mid=dogdrip&sort_index=popular&page=',
                    f'https://www.dogdrip.net/index.php?mid=dogdrip&page=',
                    f'https://www.dogdrip.net/index.php?mid=userdog&sort_index=popular&page=',
                   f'https://www.dogdrip.net/index.php?mid=userdog&page=']
        
        self.url = url_list[scrap_type] + str(page)
        
        # 게시판별 첫번째 글 순서
        index_list = [1,4,1,6]
        self.index_type = index_list[scrap_type]
        
    def chk_latest_title(self,recent_title):
        """최신글 업데이트 확인 메소드, latest.txt에 최신글 제목을 저장"""
        
        try:
            latest = open('latest.txt','r',-1,'utf-8')
        except FileNotFoundError:
            latest = open('latest.txt','w+'-1,'utf-8')
            latest.close()
            return False
        else:
            title = latest.read()
            latest.close()

            if title == recent_title:
                # 글 업데이트 확인
                return True
            else:
                print('최신 글 :',recent_title)
                latest = open('latest.txt', 'w+', -1, 'utf-8')
                latest.write(recent_title)
                return False
    
    def do_scrap(self):
        """개드립 커뮤니티 스크랩 메소드"""
        response = requests.get(self.url,headers = requests_headers)
        
        if response.status_code == 200: # 사이트 정상 응답
            
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')

            post_list = {} # 스크랩 정보

            # 게시글 제목
            title = soup.select_one(f'#main > div > div.eq.section.secontent.background-color-content > div > div > table > tbody > tr:nth-child({self.index_type}) > td.title > span > a > span.ed.title-link')
            
            if self.chk_latest_title(title.text): 
                # 글 업데이트 확인
                print("글이 올라오지 않았습니다.")
                return
            post_list['title'] = title
            
            # 게시글 링크
            if self.index_type % 2 != 0:
                link = 'https://www.dogdrip.net/' + soup.find_all('a',{'class':'ed link-reset'})[self.index_type - 1]['href']
            else:
                link = soup.find_all('a',{'class':'ed link-reset'})[self.index_type - 1]['href']
            post_list['link'] = link

            # 게시글 컨텐츠
            content_response = requests.get(link, headers = requests_headers)
            content_html = content_response.text
            content_soup = BeautifulSoup(content_html, 'html.parser')
        #     content = content_soup.select('p')
            content = content_soup.select_one('#article_1 > div')

            content_list = ""
            for i,content in enumerate(content):
                if content.next_element.name == 'img':
                    src = content.next_element['src']
                    content = '<p><img src='+'https://www.dogdrip.net' + src + '></p>' 
                content_list += str(content)
            post_list['content'] = content_list

            # html 변환 및 저장
            html_file = open('dogdrip.html', 'w+', -1, 'utf-8')
            html_file.write(content_list)
            html_file.close()

            return post_list
    
        else:
            print(response.status_code)
            return
        
    def __str__(self):
        return self.url

In [236]:
test = dogdrip_scrap(1)

In [237]:
test.do_scrap()

최신 글 : 현직 공무원이 알려주는 공문서 작성법


{'title': <span class="ed title-link">현직 공무원이 알려주는 공문서 작성법</span>,
 'link': 'https://www.dogdrip.net/319027974',
 'content': '<p><img src=https://www.dogdrip.net/dvs/d/21/04/15/c8a3026eab0c7a8e6577eb0b669ce12c.jpg></p>'}

In [157]:
# 페이지
page = '1'

# 주소
url = f'https://www.dogdrip.net/index.php?mid=dogdrip&page=' + page

# 사용자 위장 헤더
requests_headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; ; NCLIENT50_AAP5EF3046787B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
}


response = requests.get(url,headers = requests_headers)

# 개드립 인기글 스크래핑
dogdrip_scraper()

최신 글 : "한국 수도가 서울인건 상식맞지?".jpg
https://www.dogdrip.net/319011005


{'link': 'https://www.dogdrip.net/319011005',
 'content': '<p><img src=https://www.dogdrip.net/dvs/d/21/04/15/99df7b05682c93b51d86cb79e287a535.jpg></p>\n<p>\xa0</p>\n<p>\xa0</p>\n<p>\xa0</p>\n<p><img src=https://www.dogdrip.net/dvs/d/21/04/15/12a72ea219d47b68701dd0d4e2c444f1.jpg></p>\n<p>\xa0</p>'}

In [153]:
def dogdrip_scraper():
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        post_list = {}
        
        # 게시글 제목
#         title = soup.select_one('#main > div > div.eq.section.secontent.background-color-content > div > div > table > tbody > tr:nth-child(1) > td.title > span > a > span.ed.title-link')
        title = soup.select_one('#main > div > div.eq.section.secontent.background-color-content > div > div > table > tbody > tr:nth-child(4) > td.title > span > a > span.ed.title-link')
        if chk_latest_title(title.text): 
            # 글 업데이트 확인
            print("글이 올라오지 않았습니다.")
            return
        
        # 게시글 링크
#         link = 'https://www.dogdrip.net/' + soup.find_all('a',{'class':'ed link-reset'})[0]['href']
        link = soup.find_all('a',{'class':'ed link-reset'})[3]['href']
        post_list['link'] = link
        print(link)
        
        # 게시글 컨텐츠
        content_response = requests.get(link, headers = requests_headers)
        content_html = content_response.text
        content_soup = BeautifulSoup(content_html, 'html.parser')
    #     content = content_soup.select('p')
        content = content_soup.select_one('#article_1 > div')

        content_list = ""
        for i,content in enumerate(content):
            if content.next_element.name == 'img':
                src = content.next_element['src']
                content = '<p><img src='+'https://www.dogdrip.net' + src + '></p>' 
            content_list += str(content)
        post_list['content'] = content_list

        # html 변환 및 저장
        html_file = open('test.html', 'w+', -1, 'utf-8')
        html_file.write(content_list)
        html_file.close()

        return post_list
    
    else:
        print(response.status_code)
        return

In [46]:
def chk_latest_title(recent_title):
    try:
        latest = open('latest.txt','r',-1,'utf-8')
    except FileNotFoundError:
        latest = open('latest.txt','w+'-1,'utf-8')
        latest.close()
        return False
    else:
        title = latest.read()
        latest.close()
        
        if title == recent_title:
            # 글 업데이트 확인
            return True
        else:
            print('최신 글 :',recent_title)
            latest = open('latest.txt', 'w+', -1, 'utf-8')
            latest.write(recent_title)
            return False

In [ ]:
while True:
    
    
    time.sleep(30) # 30초 휴식
    

In [56]:
dogdrip_scraper()

글이 올라오지 않았습니다.


In [156]:
latest = open('latest.txt','r')
latest.close()